In [ ]:
import fgbuster
import numpy as np

freqs = np.array([85., 95., 145., 155., 220., 270.])
bandwidth = np.array([20.4, 22.8, 31.9, 34.1, 48.4, 59.4])
dnu_nu = bandwidth/freqs
beam_fwhm = np.array([25.5, 25.5, 22.7, 22.7, 13., 13.])
mukarcmin_TT = np.array([2.02, 1.78, 3.89, 4.16, 10.15, 17.4])
mukarcmin_EE = np.array([1.34, 1.18, 1.8, 1.93, 4.71, 8.08])
mukarcmin_BB = np.array([1.27, 1.12, 1.76, 1.89, 4.6, 7.89])
ell_min = np.array([30, 30, 30, 30, 30, 30])
nside = np.array([512, 512, 512, 512, 512, 512])
edges_min = freqs * (1. - dnu_nu/2)
edges_max = freqs * (1. + dnu_nu/2)
edges = [[edges_min[i], edges_max[i]] for i in range(len(freqs))]
s4_config = {
    'nbands': len(freqs),
    'frequency': freqs,
    'depth_p': 0.5*(mukarcmin_EE + mukarcmin_BB),
    'depth_i': mukarcmin_TT,
    'depth_e': mukarcmin_EE,
    'depth_b': mukarcmin_BB,
    'fwhm': beam_fwhm,
    'bandwidth': bandwidth,
    'dnu_nu': dnu_nu,
    'ell_min': ell_min,
    'nside': nside,
    'fsky': 0.03,
    'ntubes': 12,
    'nyears': 7.,
    'edges': edges,
    'effective_fraction': np.zeros(len(freqs))+1.
            }

### QUBIC Sub-optimality : values from Louise Mousset's PhD thesis
def fct_subopt(nus):
    subnus = [150., 220]
    subval = [1.4, 1.2]
    fct_subopt = np.poly1d(np.polyfit(subnus, subval, 1))
    return fct_subopt(nus)

subnus = [150., 220]
subval = [1.4, 1.2]


def qubicify(config, qp_nsubs, qp_effective_fraction):
    nbands = np.sum(qp_nsubs)
    qp_config = config.copy()
    for k in qp_config.keys():
        qp_config[k]=[]
    qp_config['nbands'] = nbands
    qp_config['fsky'] = config['fsky']
    qp_config['ntubes'] = config['ntubes']
    qp_config['nyears'] = config['nyears']
    qp_config['initial_band'] = []
    
    for i in range(len(config['frequency'])):
        #print(config['edges'][i][0], config['edges'][i][-1])
        newedges = np.linspace(config['edges'][i][0], config['edges'][i][-1], qp_nsubs[i]+1)
        #print(newedges)
        newfreqs = (newedges[0:-1]+newedges[1:])/2
        newbandwidth = newedges[1:] - newedges[0:-1]
        newdnu_nu = newbandwidth / newfreqs
        newfwhm = config['fwhm'][i] * config['frequency'][i]/newfreqs
        scalefactor_noise = np.sqrt(qp_nsubs[i]) * fct_subopt(config['frequency'][i]) / qp_effective_fraction[i]
        newdepth_p = config['depth_p'][i] * np.ones(qp_nsub[i]) * scalefactor_noise
        newdepth_i = config['depth_i'][i] * np.ones(qp_nsub[i]) * scalefactor_noise
        newdepth_e = config['depth_e'][i] * np.ones(qp_nsub[i]) * scalefactor_noise
        newdepth_b = config['depth_b'][i] * np.ones(qp_nsub[i]) * scalefactor_noise
        newell_min = np.ones(qp_nsub[i]) * config['ell_min'][i]
        newnside = np.ones(qp_nsub[i]) * config['nside'][i]
        neweffective_fraction = np.ones(qp_nsub[i]) * qp_effective_fraction[i]
        initial_band = np.ones(qp_nsub[i]) * config['frequency'][i]
        
        for k in range(qp_nsubs[i]):
            if qp_effective_fraction[i] != 0:
                qp_config['frequency'].append(newfreqs[k])
                qp_config['depth_p'].append(newdepth_p[k])
                qp_config['depth_i'].append(newdepth_i[k])
                qp_config['depth_e'].append(newdepth_e[k])
                qp_config['depth_b'].append(newdepth_b[k])
                qp_config['fwhm'].append(newfwhm[k])
                qp_config['bandwidth'].append(newbandwidth[k])
                qp_config['dnu_nu'].append(newdnu_nu[k])
                qp_config['ell_min'].append(newell_min[k])
                qp_config['nside'].append(newnside[k])
                
                qp_config['effective_fraction'].append(neweffective_fraction[k])
                qp_config['initial_band'].append(initial_band[k])
        for k in range(qp_nsubs[i]+1):
            if qp_effective_fraction[i] != 0:
                qp_config['edges'].append(newedges[k])
    fields = ['frequency', 'depth_p', 'depth_i', 'depth_e', 'depth_b', 'fwhm', 'bandwidth', 
              'dnu_nu', 'ell_min', 'nside', 'edges', 'effective_fraction', 'initial_band']
    for j in range(len(fields)):
        qp_config[fields[j]] = np.array(qp_config[fields[j]])
        
    return qp_config



qp_nsub = np.array([1, 1, 1, 1, 5, 5])
qp_effective_fraction = np.array([1, 1, 1, 1, 1, 1])
qp_config = qubicify(s4_config, qp_nsub, qp_effective_fraction)

#Corrected depths
qp_config['depth_p'][:4] = s4_config['depth_p'][:4]
qp_config['depth_i'][:4] = s4_config['depth_i'][:4]

In [ ]:
# comp=[fgbuster.component_model.CMB(), fgbuster.component_model.Dust(nu0=145, temp=20)]

# # Define Mixing Matrix from FGB 
# A = fgbuster.mixingmatrix.MixingMatrix(*comp)

# config=s4_config

# instr = fgbuster.get_instrument('CMBS4')
# instr.frequency=config['frequency']
# instr.depth_i=config['depth_i']
# instr.depth_p=config['depth_p']
# instr.fwhm=np.zeros(len(config['frequency']))

# A_ev = A.evaluator(instr.frequency)

Here you can play with the $\beta$ estimated, you'll see that the SED of dust is always 1 at reference frequency ($\nu_0 = 145$ GHz so the third line). 

In [ ]:
# A_ev([1.64])

In [ ]:
# for i in range(len(instr.frequency)):
#     print(instr.frequency[i], A_ev([1.64])[i,0], A_ev([1.64])[i,1])

In [ ]:
# beta_scalar=1.14
# sed=comp[1].eval(np.arange(10, 300, 1), beta_scalar)

# plt.figure(figsize=(20, 20))
# plt.loglog(np.arange(10, 300, 1), sed)
# plt.axhline(1)
# plt.axvline(145)
# plt.show()

Here te same for the 2 $\beta$ model, and there is a mistake somewhere.

In [ ]:
from importlib import reload
reload(fgbuster)
comp=[fgbuster.component_model.CMB(), fgbuster.component_model.Dust_2b(nu0=145, temp=20)]

# Define Mixing Matrix from FGB 
A = fgbuster.mixingmatrix.MixingMatrix(*comp)

config=s4_config

instr = fgbuster.get_instrument('CMBS4')
instr.frequency=config['frequency']
instr.depth_i=config['depth_i']
instr.depth_p=config['depth_p']
instr.fwhm=np.zeros(len(config['frequency']))

A_ev = A.evaluator(instr.frequency)

In [ ]:
A_ev([1.44, 1.64, 260])

In [ ]:
for i in range(len(instr.frequency)):
    print(instr.frequency[i], A_ev([1.44, 1.64, 260])[i,0], A_ev([1.44, 1.64, 260])[i,1])

In [ ]:
beta0_scalar=1.59
beta1_scalar=1.49
nubreak=260

nus = np.linspace(100, 300, 100)
sed=comp[1].eval(nus, beta0_scalar, beta1_scalar, nubreak)

plt.figure(figsize=(20, 20))
plt.loglog(nus, sed)
plt.axhline(1, color='k', ls=':')
plt.axvline(145, color='k', ls=':')
plt.show()

In [ ]:
from scipy import constants
from astropy.cosmology import Planck15

def my2beta(nu, beta_d0, beta_d1, nubreak, temp=20., nu0=145., doplot=False):
    Tcmb = Planck15.Tcmb(0).value
    h_over_k = constants.h * 1e9 / constants.k
    K_RJ2K_CMB = (expm1(h_over_k * nu / Tcmb)**2 / (exp(h_over_k * nu / Tcmb) * (h_over_k * nu / Tcmb)**2))
    K_RJ2K_CMB_NU0 = (expm1(h_over_k * nu0 / Tcmb)**2 / (exp(h_over_k * nu0 / Tcmb) * (h_over_k * nu0 / Tcmb)**2))
        
    fac = (1./((nu0 / nubreak)**(beta_d0-beta_d1)))
    fac2 = fac - 1
    factor = fac2 * (0.5 + 0.5*tanh((nubreak-nu0)*500))+1
    
    analytic_expr1 = factor * (exp(nu0 / temp * h_over_k) - 1) / (exp(nu / temp * h_over_k) - 1) \
                        * (nu / nu0)**(1 + beta_d0) * (nu0 / nubreak)**(beta_d0-beta_d1) \
                        * (1-(0.5 + 0.5*tanh((nu-nubreak)*500)))
    
    
    analytic_expr2 = factor * (exp(nu0 / temp * h_over_k) - 1) / (exp(nu / temp * h_over_k) - 1) \
                          * (nu / nu0)**(1 + beta_d1) * (0.5 + 0.5*tanh((nu-nubreak)*500)) 
    
    
    ### CMB
    # analytic_expr = analytic_expr1 * K_RJ2K_CMB_NU0 + analytic_expr2 * K_RJ2K_CMB_NU0
    ### RJ
    analytic_expr = analytic_expr1 + analytic_expr2
    
    if doplot:
        plot(nu, analytic_expr1, lw=6)
        plot(nu, analytic_expr2, lw=6)
        plot(nu, analytic_expr, lw=6, ls='--')
        axvline(x=nu0)
        axhline(y=1)
    
    return analytic_expr
    
beta0=1.9
beta1=1.2
nubreak=260.
nu0 = 145.
  
nus = np.linspace(100, 300, 100)
a=my2beta(nus, beta0, beta1, nubreak, doplot=True, nu0=nu0)
#print(my2beta(145, beta0, beta1, nubreak, doplot=True))

In [ ]:
vv = 200.
plot(nus, (vv-1)*(0.5 + 0.5*tanh((nubreak-nus)*500))+1)
print((vv-1)*(0.5 + 0.5*tanh((nubreak-nus)*500))+1)